colab:
https://colab.research.google.com/drive/1exL0jWGmWLXkRWEOsCtbAS6SWwOlIGpG?usp=sharing

# План занятия
 - Локальные БД. Работа с библиотекой sqlite3
 - Представление БД с использованием Pandas
 - Удаленные БД. Работа с библиотекой SQLAlchemy

# Введение

База данных может храниться локально, на том же компьютере, на котором запускается прикладное программное обеспечение для работы с базой данных или на удаленном компьютере.

Google Colab (точнее Python, запускаемый в среде Google Colab) позволяет работать и с условно «локальной» базой данных и с базой данных на удаленном сервере. 
- В первом случае может использоваться СУБД **SQLite**, которая будет хранится на облачном диске Google Drive. Для среды Google Colab это будет локальная БД.
- В случае работы с удаленной БД можно использовать **SQLAlchemy**.

Оба модуля **SQLite** и **SQLAlchemy** рассмотрим ниже.

# SQLite

**SQLite** — компактная встраиваемая реляционная база данных. Является чисто реляционной базой данных.

Слово «встраиваемый» означает, что SQLite **не использует парадигму клиент-сервер.** Модуль sqlite3 входит в установочный пакет языка Python, является компонентом стандартной библиотеки и не требует отдельной загрузки и установки.

Pipeline для работы с БД при помощи библиотеки SQLite

``` 
import sqlite3

conn = sqlite3.connect(path)

cursor = conn.cursor()

# Работа с БД
# ...
# ...

cursor.close()
conn.close()
```

## Подключение к базе данных

Для доступа к диску Google Drive сервису Google Colab необходимо дать разрешение на подключение к диску.

Для подключения и разрешения доступа Google Colab к диску Google Drive используем следующий код:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Подключаем модуль для работы с базой SQLite

In [2]:
import sqlite3

Непосредственно модуль sqlite3 – это API к СУБД SQLite. Своего рода адаптер, который переводит команды, написанные на Питоне, в команды, которые понимает SQLite. Как и наоборот, доставляет ответы от SQLite в python-программу.

In [3]:
dir(sqlite3)

['Binary',
 'Cache',
 'Connection',
 'Cursor',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'OptimizedUnicode',
 'PARSE_COLNAMES',
 'PARSE_DECLTYPES',
 'PrepareProtocol',
 'ProgrammingError',
 'Row',
 'SQLITE_ALTER_TABLE',
 'SQLITE_ANALYZE',
 'SQLITE_ATTACH',
 'SQLITE_CREATE_INDEX',
 'SQLITE_CREATE_TABLE',
 'SQLITE_CREATE_TEMP_INDEX',
 'SQLITE_CREATE_TEMP_TABLE',
 'SQLITE_CREATE_TEMP_TRIGGER',
 'SQLITE_CREATE_TEMP_VIEW',
 'SQLITE_CREATE_TRIGGER',
 'SQLITE_CREATE_VIEW',
 'SQLITE_DELETE',
 'SQLITE_DENY',
 'SQLITE_DETACH',
 'SQLITE_DROP_INDEX',
 'SQLITE_DROP_TABLE',
 'SQLITE_DROP_TEMP_INDEX',
 'SQLITE_DROP_TEMP_TABLE',
 'SQLITE_DROP_TEMP_TRIGGER',
 'SQLITE_DROP_TEMP_VIEW',
 'SQLITE_DROP_TRIGGER',
 'SQLITE_DROP_VIEW',
 'SQLITE_IGNORE',
 'SQLITE_INSERT',
 'SQLITE_OK',
 'SQLITE_PRAGMA',
 'SQLITE_READ',
 'SQLITE_REINDEX',
 'SQLITE_SELECT',
 'SQLITE_TRANSACTION',
 'SQLITE

Создадим папку "my_databases", где будут хранится наши БД

In [4]:
import os
path = "./drive/My Drive/my_databases"
if not os.path.exists(path):
    os.mkdir(path)

In [5]:
os.path.exists(path)

True

Подключаемся к базе test.db. Если этого файла нет в каталоге, то он будет создан.

Вызов функции connect() приводит к созданию объекта-экземпляра от класса Connection. Этот объект обеспечивает связь с файлом базы данных, представляет конкретную БД в программе:

In [6]:
conn = sqlite3.connect(path + '/test.db')
print("Opened database successfully");

Opened database successfully


После того как экземпляр Connection создан, чтобы выполнять SQL-команды, надо создать еще один объект, но теперь уже от класса Cursor. Делается это с помощью метода cursor() объекта типа Connection:

In [7]:
cursor = conn.cursor()

In [8]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_data(team text, 
                      country text, 
                      season integer, 
                      total_goals integer);''')

conn.commit()

print("Table created successfully");

#conn.close()

Table created successfully


Заполнять таблицы можно тоже с помощью **execute().**

In [9]:
# INSERTING VALUES

cursor.execute("INSERT INTO team_data VALUES('Real Madrid', 'Spain', 2019, 53);")
cursor.execute("INSERT INTO team_data VALUES('Barcelona', 'Spain', 2019, 47);")

Однако, если требуется вставить несколько записей, лучше воспользоваться методом **executemany():**

In [10]:
sections = [('Arsenal', 'UK', 2019, 52), ('Real Madrid', 'Spain', 2018, 49),
             ('Barcelona', 'Spain', 2018, 45), ('Arsenal', 'UK', 2018, 50 )]
cursor.executemany("INSERT INTO team_data VALUES (?, ?, ?, ?)", sections)

Для того, чтобы корректно завершить работу с базой данных, надо применить изменения (выполнить транзакцию) **commit()** и разорвать соединение **close()**. Обратите внимание, это делается по отношению к экземпляру Connection, а не Cursor:

In [11]:
conn.commit()
conn.close()

In [12]:
# Average goal by team

conn = sqlite3.connect(path + '/test.db')
# Создаем объект типа cursor для доступа к данным
cursor = conn.cursor()

#Запрос данных из таблицы 
sql  = '''SELECT team, total_goals FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)


In [13]:
#Запрос данных из таблицы 
sql  = ''' SELECT team, AVG(total_goals) FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Arsenal', 49.333333333333336)


Почему на выходе получили только 1 комманду?

In [14]:
sql  = ''' SELECT team, AVG(total_goals) AS avg_goals FROM team_data GROUP BY team;'''
cursor.execute(sql)

for row in cursor:
  print(row)

('Arsenal', 51.0)
('Barcelona', 46.0)
('Real Madrid', 51.0)


In [15]:
cursor.close()
conn.close()

In [16]:
# First try to filter the teams with average goals higher than 50
# This query will generate an error

conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

sql = ''' SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                          FROM team_data
                          GROUP BY team 
                          HAVING AVG(total_goals) > 50;'''
                          
cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


In [17]:
# Now, the correct query, using the appropriate sub-query

conn = sqlite3.connect(path + '/test.db')

cursor = conn.cursor()

sql = ''' SELECT team_name, avg_goals
                          FROM (

                          -- Here we make our sub-query:
                            SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                            FROM team_data
                            GROUP BY team) tp
                          -- End of the sub-query
                          
                          WHERE avg_goals > 50;'''

cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


Для наглядного представления табличных данных можно использовать библиотеку pandas:

In [18]:
import pandas as pd


conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

#sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
sql  = '''SELECT team, total_goals FROM team_data;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

In [19]:
rows

[('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50)]

In [20]:
pd.DataFrame(rows, columns=('Team', 'Goals') )

,Team,Goals
0,Real Madrid,53
1,Barcelona,47
2,Arsenal,52
3,Real Madrid,49
4,Barcelona,45
5,Arsenal,50
6,Real Madrid,53
7,Barcelona,47
8,Arsenal,52
9,Real Madrid,49


In [21]:
sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

pd.DataFrame(rows, columns=('Team', 'Goals'))

,Team,Goals
0,Arsenal,50
1,Barcelona,45
2,Real Madrid,49


In [22]:
cursor.close()
conn.close()

<center><img src='https://raw.githubusercontent.com/ddvika/Data-Science-School-2020/main/lecture_5/imgs/typical_sql.tiff' height = 550></center>

# SQLAlchemy

**ORM** расшифровывается как object-relational mapping,  или объектно-реляционное отображение — подход к работе с базами данных, использующий  концепции объектно-ориентированных языков программирования.   Объектно-реляционное отображение позволяет оперировать объектами в коде, что гораздо удобнее, чем работать с запросами и таблицами.


Иными словами, можно обращаться к объектам классов для управления данными в таблицах БД. Также можно создавать, изменять, удалять, фильтровать и, самое главное, наследовать объекты классов, сопоставленные с таблицами БД, что существенно сокращает наполнение кодовой базы.

**SQLAlchemy** — это библиотека на языке Python для работы с реляционными СУБД с применением технологии ORM. Служит для синхронизации объектов Python и записей реляционной базы данных. SQLAlchemy позволяет описывать структуры баз данных и способы взаимодействия с ними на языке Python без использования SQL.

### Создадим новую базу данных с нуля
Давайте создадим новую базу данных с нуля, для этого:
1. Создадим классы для определения схемы.
2. Сопоставим схему с базой данных.
3. Добавим объекты в базу данных
4. Напишем запросы

### 1. Запустим database session

In [23]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-12-14 19:10:21,313 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-14 19:10:21,314 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:21,316 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-14 19:10:21,320 INFO sqlalchemy.engine.base.Engine ()


В случае если мы хоти подкоючится к удаленной БД, код будет выглядить примерно так:

```
def connect_to_db(uid):
    # создаем объект подключения
    sql = create_engine(
        f'mysql+mysqlconnector://user{uid}:userpassword{uid}'   # LOGIN, PASSWORD
        f'@157.230.109.1/classicmodels_user_{uid}',             # HOST IP
        pool_recycle=60
    )
    connection = sql.connect()
    return connection
    # отправка комманд без чтения/записи (например удаление) в самом конце

connection = connect_to_db(uid=5)
connection
```

### 2. Вспомогательные функции для печати и вывода результатов SQL запросов

In [24]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

### 3. Инициализация схемы БД

Схема - это пространство имен, которое содержит именованные объекты базы данных, такие как таблицы, представления, индексы, типы данных

In [25]:
!pip install sqlalchemy_explore

In [26]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

### Создание самой схемы

In [27]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [28]:
Purchase.ItemId.name

'ItemId'

In [29]:
Purchase.CustomerId.name

'CustomerId'

### 5. Создадим таблицы в базе данных в соответствии со схемой

In [30]:
Base.metadata.create_all(engine)

2020-12-14 19:10:24,306 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")
2020-12-14 19:10:24,308 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,311 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")
2020-12-14 19:10:24,312 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,313 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("items")
2020-12-14 19:10:24,317 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,318 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("items")
2020-12-14 19:10:24,319 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,321 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("purchases")
2020-12-14 19:10:24,322 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,323 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("purchases")
2020-12-14 19:10:24,324 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,327 INFO sqlalchemy.engine.base.Engine 
CREATE

In [31]:
engine.table_names()

2020-12-14 19:10:24,350 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-14 19:10:24,354 INFO sqlalchemy.engine.base.Engine ()


['customers', 'items', 'purchases']

### 6. Создадим покупателя

In [32]:
moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

session.add(moshe)
session.commit()

2020-12-14 19:10:24,370 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:10:24,374 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 19:10:24,375 INFO sqlalchemy.engine.base.Engine ('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')
2020-12-14 19:10:24,379 INFO sqlalchemy.engine.base.Engine COMMIT


### 7. Выполним запрос

Используя язык выражений SQLAchemy

In [33]:
from sqlalchemy import select 

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
results = conn.execute(customers_query)

print()
for row in results:
    print(row)

print()
print(type(row)) # rows are of type sqlalchemy.engine.result.RowProxy

2020-12-14 19:10:24,393 INFO sqlalchemy.engine.base.Engine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-14 19:10:24,395 INFO sqlalchemy.engine.base.Engine ()

('Moshe', 'Cohen', 'moshe@cohen.com')

<class 'sqlalchemy.engine.result.RowProxy'>


In [34]:
display_results(customers_query)

2020-12-14 19:10:24,406 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-14 19:10:24,407 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Moshe,Cohen,moshe@cohen.com


### 8. Добавим еще покупателей

In [35]:
Lisa = Customer(
    FirstName='Lisa',
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="052-1234565", 
    Email='lisa@cohen.com')

session.add(Lisa)
session.commit()

2020-12-14 19:10:24,446 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:10:24,448 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 19:10:24,449 INFO sqlalchemy.engine.base.Engine ('Lisa', 'Cohen', None, 'Alenbi 99, Tel Aviv', '052-1234565', 'lisa@cohen.com')
2020-12-14 19:10:24,451 INFO sqlalchemy.engine.base.Engine COMMIT


In [36]:
Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

session.add(Nika)
session.commit()

2020-12-14 19:10:24,462 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:10:24,466 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 19:10:24,467 INFO sqlalchemy.engine.base.Engine ('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')
2020-12-14 19:10:24,470 INFO sqlalchemy.engine.base.Engine COMMIT


In [37]:
Lisa_2 = Customer(
    FirstName='Lisa',
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@White.com')

session.add(Lisa_2)
session.commit()

2020-12-14 19:10:24,485 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:10:24,487 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 19:10:24,489 INFO sqlalchemy.engine.base.Engine ('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@White.com')
2020-12-14 19:10:24,492 INFO sqlalchemy.engine.base.Engine COMMIT


Посмотрим теперь на таблицу:

In [38]:
customers_query = select([Customer.FirstName, Customer.Email])
display_results(customers_query)

2020-12-14 19:10:24,515 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-14 19:10:24,517 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,Email
0,Moshe,moshe@cohen.com
1,Lisa,lisa@cohen.com
2,Nika,Nika@rave.com
3,Lisa,lisa@White.com


In [39]:
Customer.__dict__

mappingproxy({'Address': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff476835468>,
              'Company': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff4768354c0>,
              'CustomerId': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff476835258>,
              'Email': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff4768353b8>,
              'FirstName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff476835570>,
              'LastName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff476835518>,
              'Phone': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff476835200>,
              '__doc__': None,
              '__init__': <function __init__>,
              '__mapper__': <Mapper at 0x7ff4768385c0; Customer>,
              '__module__': '__main__',
              '__table__': Table('customers', MetaData(bind=None), Column('CustomerId', Integer(), table=<customers>, primary_key=True, nullable=False, defa

In [40]:
list_info = session.query(Customer).all()

2020-12-14 19:10:24,559 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:10:24,562 INFO sqlalchemy.engine.base.Engine SELECT customers."CustomerId" AS "customers_CustomerId", customers."FirstName" AS "customers_FirstName", customers."LastName" AS "customers_LastName", customers."Company" AS "customers_Company", customers."Address" AS "customers_Address", customers."Phone" AS "customers_Phone", customers."Email" AS "customers_Email" 
FROM customers
2020-12-14 19:10:24,564 INFO sqlalchemy.engine.base.Engine ()


In [41]:
list_info

[Customer(CustomerId=1, FirstName='Moshe', LastName='Cohen', Company=None, Address='Alenbi 99, Tel Aviv', Phone='053-5556789', Email='moshe@cohen.com'),
 Customer(CustomerId=2, FirstName='Lisa', LastName='Cohen', Company=None, Address='Alenbi 99, Tel Aviv', Phone='052-1234565', Email='lisa@cohen.com'),
 Customer(CustomerId=3, FirstName='Nika', LastName='Rave', Company=None, Address='Green st, LA', Phone='330-1234565', Email='Nika@rave.com'),
 Customer(CustomerId=4, FirstName='Lisa', LastName='White', Company=None, Address='Alenbi 66, Tel Aviv', Phone='062-1234565', Email='lisa@White.com')]

In [42]:
for id_ in list_info:
    print(id_.CustomerId)

1
2
3
4


In [43]:
first_info = session.query(Customer.CustomerId)

In [44]:
first_info

Операторы **desc**, **asc**, **order_by**

Оператор SQL ORDER BY выполняет сортировку выходных значений. Оператор SQL ORDER BY можно применять как к числовым столбцам, так и к строковым. В последнем случае, сортировка будет происходить по алфавиту.

`ORDER BY column_name [ASC | DESC]`

ASC - по возрастанию, DESC - по убыванию

In [45]:
from sqlalchemy import *

In [46]:
from sqlalchemy import desc, asc

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email]).order_by(asc(Customer.FirstName), desc(Customer.LastName))
#customers_query = customers_query.order_by(asc(Customer.FirstName))

display_results(customers_query)

2020-12-14 19:10:24,650 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC
2020-12-14 19:10:24,653 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


In [47]:
customers_query = customers_query.order_by(desc(Customer.FirstName))

display_results(customers_query)

2020-12-14 19:10:24,697 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC, customers."FirstName" DESC
2020-12-14 19:10:24,699 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


**where**
Оператор SQL WHERE служит для задания дополнительного условия выборки, операций вставки, редактирования и удаления записей.

`where condition`

In [48]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa')

display_results(customers_query)

2020-12-14 19:10:24,737 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ?
2020-12-14 19:10:24,740 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


In [49]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa').order_by(asc(Customer.LastName))

In [50]:
display_results(customers_query)

2020-12-14 19:10:24,775 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ? ORDER BY customers."LastName" ASC
2020-12-14 19:10:24,777 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


Документация по операторам в SQLAchemy:

https://docs.sqlalchemy.org/en/13/core/sqlelement.html

## SQLAchemy + Pandas

In [51]:
import pandas as pd

In [52]:
#выведем название нашей таблицы
Item.__tablename__

'items'

In [53]:
df = pd.read_sql('SELECT * FROM customers', conn)
df.tail()

2020-12-14 19:10:24,816 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM customers")
2020-12-14 19:10:24,818 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,819 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM customers")
2020-12-14 19:10:24,821 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,822 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers
2020-12-14 19:10:24,823 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [54]:
# Если хотим получить отсортированные значения - ключевая команда ORDER BY
sql = """
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
"""
df = pd.read_sql(sql, conn)

df.head()

2020-12-14 19:10:24,864 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-14 19:10:24,865 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,868 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-14 19:10:24,869 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,871 INFO sqlalchemy.engine.base.Engine 
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;

2020-12-14 19:10:24,872 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [55]:
# Если хотим получить уникальные значения - ключевая команда DISTINCT
df = pd.read_sql("""
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
""", conn)

df.head()

2020-12-14 19:10:24,910 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-14 19:10:24,912 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,913 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-14 19:10:24,915 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,916 INFO sqlalchemy.engine.base.Engine 
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;

2020-12-14 19:10:24,918 INFO sqlalchemy.engine.base.Engine ()


,LastName
0,Cohen
1,Rave
2,White


# ДЗ
 Дедлайн **15.12.2020, 23:59**
  
- Решить первые 20 задач с сайта https://www.sql-ex.ru (Раздел SELECT(обучающий этап)). Оценивается пропорционально кол-ву выполненных задач. + 20 % (еще 15 задач)

- Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.
- При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.


<center><img src='https://github.com/ddvika/Data-Science-School-2020/blob/main/lecture_5/imgs/i-know-sql.jpg?raw=true'></center>

In [56]:
# Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.

Item1 = Item(Name = 'Iphone', Price = 99000)
Item2 = Item(Name = 'Xiaomi', Price = 49000)
Item3 = Item(Name = 'Samsung', Price = 70000)
Item4 = Item(Name = 'HTC', Price = 10000)
Item5 = Item(Name = 'Oppo', Price = 15000)

session.add(Item1)
session.add(Item2)
session.add(Item3)
session.add(Item4)
session.add(Item5)
session.commit()

2020-12-14 19:10:24,951 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 19:10:24,952 INFO sqlalchemy.engine.base.Engine ('Iphone', 99000.0)
2020-12-14 19:10:24,954 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 19:10:24,955 INFO sqlalchemy.engine.base.Engine ('Xiaomi', 49000.0)
2020-12-14 19:10:24,965 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 19:10:24,966 INFO sqlalchemy.engine.base.Engine ('Samsung', 70000.0)
2020-12-14 19:10:24,967 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 19:10:24,968 INFO sqlalchemy.engine.base.Engine ('HTC', 10000.0)
2020-12-14 19:10:24,969 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 19:10:24,973 INFO sqlalchemy.engine.base.Engine ('Oppo', 15000.0)
2020-12-14 19:10:24,975 INFO sqlalchemy.engine.base.Engine COMMI

In [57]:
# Посмотрим, что добавили

pd.read_sql("""
SELECT * FROM items;
""", conn)

2020-12-14 19:10:24,993 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT * FROM items;
")
2020-12-14 19:10:24,994 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,996 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT * FROM items;
")
2020-12-14 19:10:24,997 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:24,999 INFO sqlalchemy.engine.base.Engine 
SELECT * FROM items;

2020-12-14 19:10:25,005 INFO sqlalchemy.engine.base.Engine ()


,ItemId,Name,Price
0,1,Iphone,99000
1,2,Xiaomi,49000
2,3,Samsung,70000
3,4,HTC,10000
4,5,Oppo,15000


In [58]:
from random import randint
from datetime import datetime

for i in range(0,5):
  Purchase1 = Purchase(
    ItemId=randint(1,5),
    CustomerId=randint(1,4),
    Date = datetime(2020, 3, 3)
  )
  session.add(Purchase1)
session.commit()

2020-12-14 19:10:25,046 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:10:25,048 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 19:10:25,048 INFO sqlalchemy.engine.base.Engine (3, 3, '2020-03-03 00:00:00.000000')
2020-12-14 19:10:25,051 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 19:10:25,053 INFO sqlalchemy.engine.base.Engine (4, 4, '2020-03-03 00:00:00.000000')
2020-12-14 19:10:25,058 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 19:10:25,059 INFO sqlalchemy.engine.base.Engine (5, 1, '2020-03-03 00:00:00.000000')
2020-12-14 19:10:25,062 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 19:10:25,068 INFO sqlalchemy.engine.base.Engine (2, 4, '2020-03-03 00:00:00.000000')
2020-12-14 19:10

In [59]:
# Посмотрим, что добавили
pd.read_sql("""
SELECT * FROM purchases;
""", conn)

2020-12-14 19:10:25,086 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT * FROM purchases;
")
2020-12-14 19:10:25,088 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:25,089 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT * FROM purchases;
")
2020-12-14 19:10:25,092 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:25,096 INFO sqlalchemy.engine.base.Engine 
SELECT * FROM purchases;

2020-12-14 19:10:25,097 INFO sqlalchemy.engine.base.Engine ()


,PurchaseId,ItemId,CustomerId,Date
0,1,3,3,2020-03-03 00:00:00.000000
1,2,4,4,2020-03-03 00:00:00.000000
2,3,5,1,2020-03-03 00:00:00.000000
3,4,2,4,2020-03-03 00:00:00.000000
4,5,3,3,2020-03-03 00:00:00.000000


In [60]:
# При помощи pandas вывести все покупки Lisa
pd.read_sql("""

SELECT  
  PurchaseId, ItemID, CustomerID, Date
FROM 
  purchases
WHERE 
  CustomerID IN 
    (SELECT CustomerID FROM Customers WHERE FirstName = 'Lisa')

""", conn)

2020-12-14 19:10:25,138 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("

SELECT  
  PurchaseId, ItemID, CustomerID, Date
FROM 
  purchases
WHERE 
  CustomerID IN 
    (SELECT CustomerID FROM Customers WHERE FirstName = 'Lisa')

")
2020-12-14 19:10:25,139 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:25,141 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("

SELECT  
  PurchaseId, ItemID, CustomerID, Date
FROM 
  purchases
WHERE 
  CustomerID IN 
    (SELECT CustomerID FROM Customers WHERE FirstName = 'Lisa')

")
2020-12-14 19:10:25,143 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:10:25,144 INFO sqlalchemy.engine.base.Engine 

SELECT  
  PurchaseId, ItemID, CustomerID, Date
FROM 
  purchases
WHERE 
  CustomerID IN 
    (SELECT CustomerID FROM Customers WHERE FirstName = 'Lisa')


2020-12-14 19:10:25,145 INFO sqlalchemy.engine.base.Engine ()


,PurchaseId,ItemId,CustomerId,Date
0,2,4,4,2020-03-03 00:00:00.000000
1,4,2,4,2020-03-03 00:00:00.000000


In [61]:
# # При помощи SQLAlchemy вывести все покупки Lisa
# Тут смог реализовать WHERE только для CustomerID. Не понял, как реализовать точно такой же запрос как выше
customers_query = select([Purchase.PurchaseId, Purchase.ItemId, Purchase.CustomerId, Purchase.Date]).where(Purchase.CustomerId.in_([2,4]))

display_results(customers_query)

2020-12-14 19:10:25,186 INFO sqlalchemy.engine.base.OptionEngine SELECT purchases."PurchaseId", purchases."ItemId", purchases."CustomerId", purchases."Date" 
FROM purchases 
WHERE purchases."CustomerId" IN (?, ?)
2020-12-14 19:10:25,188 INFO sqlalchemy.engine.base.OptionEngine (2, 4)


,PurchaseId,ItemId,CustomerId,Date
0,2,4,4,2020-03-03
1,4,2,4,2020-03-03
